In [4]:
with open(f"./small_dgemv_naive_c_implementation_ver3-0_ZmmNaive.c", "w") as p:

    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("#include <xmmintrin.h>\n")     

    code = f"""        
double sum_zmm_elements(__m512d zmmX){{
    __m256d ymm_upper = _mm512_extractf64x4_pd(zmmX, 1);
    __m256d ymm_lower = _mm512_extractf64x4_pd(zmmX, 0);

    __m256d ymm_sum = _mm256_add_pd(ymm_upper, ymm_lower);

    __m128d xmm_sum = _mm_add_pd(_mm256_extractf128_pd(ymm_sum, 1), _mm256_extractf128_pd(ymm_sum, 0));

    __m128d xmm_high_low = _mm_add_pd(xmm_sum, _mm_unpackhi_pd(xmm_sum, xmm_sum));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}}    
    
    

void mydgemv_n_ver3_0(double a[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m512d zmm0, zmm1, zmm2;
    double tmp_x;
    for (int64_t i=0; i<lda; i++){{
        y[i] = 0.0;
    }}
    
    int64_t i_remain = lda % 8;
    for (int64_t k=0; k<ldx; k++){{
        double tmp_x = x[k];
        zmm0 = _mm512_set1_pd(tmp_x);
        for (int64_t i=0; i<lda-i_remain; i+=8){{
            zmm1 = _mm512_loadu_pd(&y[i]);
            zmm2 = _mm512_loadu_pd(&a[i + k*lda]);
            
            zmm2 = _mm512_mul_pd(zmm0, zmm2);
            
            zmm1 = _mm512_add_pd(zmm1, zmm2);
            
            _mm512_storeu_pd(&y[i], zmm1);
        }}
        
        for (int64_t i=lda-i_remain; i<lda; i++){{
            y[i] += a[i + k*lda] * tmp_x;
        }}
    }}
}}


void mydgemv_t_ver3_0(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m512d zmm0, zmm1, zmm2;
    
    int64_t k_remain = ldx % 8;
    for (int64_t i=0; i<lda; i++){{
        zmm0 = _mm512_setzero_pd();
        
        for (int64_t k=0; k<ldx-k_remain; k+=8){{
            zmm1 = _mm512_loadu_pd(&x[k]);
            zmm2 = _mm512_loadu_pd(&a_t[k + i*ldx]);
            
            zmm1 = _mm512_mul_pd(zmm1, zmm2);
            
            zmm0 = _mm512_add_pd(zmm0, zmm1);
        }}
        
        y[i] = sum_zmm_elements(zmm0);
    }}
}}
    """
    p.write(code)